In [ ]:
from bloqade.gemini import logical as gemini_logical
from bloqade import qubit, squin
from bloqade.lanes.logical_mvp import (
    compile_to_physical_stim_program,
    kernel,
    set_detector,
    set_observable,
    TesseractDecoder,
    GeminiLogical,
)
import numpy as np
from scipy.linalg import block_diag

In [ ]:
@kernel
def main():
    # see arXiv: 2412.15165v1, Figure 3a
    reg = qubit.qalloc(5)
    squin.broadcast.t(reg)

    squin.broadcast.sqrt_x([reg[0], reg[1], reg[4]])
    squin.broadcast.cz([reg[0], reg[2]], [reg[1], reg[3]])
    squin.broadcast.sqrt_y([reg[0], reg[3]])
    squin.broadcast.cz([reg[0], reg[3]], [reg[2], reg[4]])
    squin.sqrt_x_adj(reg[0])
    squin.broadcast.cz([reg[0], reg[1]], [reg[4], reg[3]])
    squin.broadcast.sqrt_y_adj(reg)

    measurements = gemini_logical.terminal_measure(reg)

    for i in range(len(reg)):
        set_detector([measurements[i]])
        set_observable(measurements[i], i)

In [ ]:
service = GeminiLogical()
future = service.submit(main, shots=100)
results = future.get_results()

In [ ]:
results.logical_bits

In [ ]:
decoder = TesseractDecoder(results.detector_error_model)

In [ ]:
corrected_logical_bits = decoder.decode(results.detector_bits) ^ results.logical_bits
corrected_logical_bits

## Underlying pipeline

In [ ]:
program = compile_to_physical_stim_program(main)

In [ ]:
print(program)

In [ ]:
program.diagram("timeline-svg", height=400)

In [ ]:
dem = program.detector_error_model(approximate_disjoint_errors=True)
dem

In [ ]:
sampler = program.compile_detector_sampler()

In [ ]:
dets, obs = sampler.sample(shots=100_000, separate_observables=True)
dets

## Measurement-to-detector matrices

Alternatively, specify matrices `m2obs` and `m2dets` that convert measurements to detectors and observables, respectively.

`detector_bits = measurement_bits @ m2dets`

`observable_bits = measurement_bits @ m2obs`



In [ ]:
d = np.array([[1,1,1,1,0,0,0], [0,1,1,0,1,1,0], [0,0,1,1,1,0,1]])
o = np.array([[1,1,0,0,0,1,0]])

m2obs = block_diag(*[o.T]*5)  # shape (35, 5) - 35 measurements, 5 observables
m2dets = block_diag(*[d.T]*5)  # shape (35, 15) - 35 measurements, 15 detectors

In [ ]:
@kernel
def main():
    reg = qubit.qalloc(5)
    squin.broadcast.t(reg)

    squin.broadcast.sqrt_x([reg[0], reg[1], reg[4]])
    squin.broadcast.cz([reg[0], reg[2]], [reg[1], reg[3]])
    squin.broadcast.sqrt_y([reg[0], reg[3]])
    squin.broadcast.cz([reg[0], reg[3]], [reg[2], reg[4]])
    squin.sqrt_x_adj(reg[0])
    squin.broadcast.cz([reg[0], reg[1]], [reg[4], reg[3]])
    squin.broadcast.sqrt_y_adj(reg)


In [ ]:
service = GeminiLogical()
future = service.submit(main, m2obs=m2obs, m2dets=m2dets, shots=100)
results = future.get_results()

In [ ]:
print(results.phyiscal_program)

## Memory Experiment

In [ ]:
@kernel
def memory_main():
    reg = qubit.qalloc(5)
    data = reg[0]
    aux_x = [reg[1], reg[3]]
    aux_z = [reg[2], reg[4]]

    num_rounds = 2

    squin.broadcast.h(aux_x)
    for i in range(num_rounds):  # stabilizer rounds
        squin.cnot(aux_x[i], data)
        squin.cnot(data, aux_z[i])
    squin.broadcast.h(aux_x)
    
    measurements = gemini_logical.terminal_measure(reg)
    m_data = measurements[0]
    m_aux_x = [measurements[1], measurements[3]]
    m_aux_z = [measurements[2], measurements[4]]

    # detector and observable annotations
    set_observable(m_data)

    # for stabilizer readout, detectors compare stabilizer of two subsequent rounds
    set_detector([m_aux_z[0]])
    for i in range(num_rounds - 1):
        set_detector([m_aux_z[i], m_aux_z[i + 1]])
        set_detector([m_aux_x[i], m_aux_x[i + 1]])
    set_detector([m_data, m_aux_z[-1]])